In [1]:
import os
import sys

__file__ = "xo.ipynb"

def parent_dir(n=1):
    """default n=0 means current directory"""
    path = os.path.dirname(os.path.realpath(__file__))
    for _ in range(n):
        path = os.path.dirname(path)
    return path

if parent_dir() not in sys.path:
    sys.path.append(parent_dir())

In [2]:
import numpy as np

from tools.utils import *
from tools.problem import Problem, Node

In [3]:

class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))

In [4]:
class XO(Game):
    def __init__(self, initial) -> None:
        self.initial = initial
    
    def actions(self, state):
        n, m = state.shape
        actions_list = []
        for i in range(n):
            for j in range(m):
                if state[i][j] == "":
                    actions_list.append((i, j))
        return actions_list
    
    def result(self, state, action):
        new_state = state.copy()
        chr, i, j = action
        new_state[i, j] = chr
        return new_state
    
    def terminal_test(self, state) -> bool:
        for situ in self.all_situations(state):
            if np.all(situ == situ[0]):
                return True
        return not self.actions(state)
    
    def utility(self, state, player:str):
        enemy = {"x": "o", "o": "x"}
        for situation in self.all_situations(state):
            if np.all(situation == player):
                return 1
            elif np.all(situation == enemy[player]):
                return -1
        return 0
    
    def all_situations(self, state):
        situations = []
        situations.append(self.qotre_asli(state))
        situations.append(self.qotre_farei(state))
        n = state.shape[0]
        for i in range(n):
            situations.append(self.get_row(state, i))
            situations.append(self.get_column(state, i))
        return situations
    
    def get_row(self, board, n=0):
        return board[n, :]

    def get_column(self, board, n=0):
        return board[:, n]

    def qotre_asli(self, board):
        qotr = [board[i, i] for i in range(board.shape[0])]
        return np.array(qotr, dtype=str)

    def qotre_farei(self, board):
        n = board.shape[0]
        i, j = 0, n-1
        qotr = [board[i+k, j-k] for k in range(n)]
        return np.array(qotr, dtype=str)
    

In [5]:
board = np.zeros((3,3), dtype=str)

board[0, 0] = "x"
board[0, 1] = "o"
board[0, 2] = "x"
board[1, 0] = "o"
board[1, 1] = "x"
board[1, 2] = "o"
board[2, 0] = "x"
board[2, 1] = "o"
board[2, 2] = "x"

board = [
    ["x", "", "x"],
    ["o", "o", "o"],
    ["x", "", ""],
]
board = np.array(board, dtype=str)

print(board)
game = XO(board)

game.utility(board, "x")
game.terminal_test(board)

[['x' '' 'x']
 ['o' 'o' 'o']
 ['x' '' '']]


True